<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 11:17:34
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.40 C
-------------------
Today PnL: 50.68 K (0.36%)
Current PnL: -17.53 L (-11.79%)
CY Booked + Current PnL: -8.49 L (-5.71%)
-------------------
Total profit:  3.51 L
Total loss:  -21.04 L
-------------------
Total Booked + Current PnL: 18.49 L (15.2%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.47%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.20 L (56.69%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-15.77,65.0,H-LC,8.54,116384.0,12072.0,4574.0,0.89,11.57,3.93,15.96,9.0,2.64,0.85,30.43,X5K,ATH,METALS
53,MEDANTA,1486.00,16.52,65.0,H-MC,15.65,166425.0,40225.0,5958.0,3.19,31.87,3.58,36.59,108.0,6.75,1.21,48.44,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.60,68.0,H-MC,6.78,140824.0,25888.0,10703.0,0.28,22.52,7.60,31.83,81.0,2.42,1.02,45.97,X40N,ATH,IT
82,TTKPRESTIG,770.00,107.68,71.0,M-SC,6.97,89434.0,-11343.0,11439.0,0.11,-11.26,12.79,0.09,245.0,-0.99,0.65,17.50,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.86,57.0,H-LC,8.93,170027.0,13850.0,12242.0,0.61,8.87,7.20,16.71,22.0,1.13,1.24,27.85,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,5.18,45.0,H-SC,11.09,125433.0,136.0,58866.0,1.16,0.11,46.93,47.09,141.0,0.00,0.91,56.99,MH,ATH,POWER
39,ICICIPRULI,790.00,-20.54,40.0,H-MC,2.16,136820.0,1032.0,41716.0,-0.86,0.76,30.49,31.48,107.0,0.02,0.99,12.97,X40,ATH,INSURANCE
32,HAVELLS,2069.16,-0.51,62.0,H-MC,2.30,246730.0,-1001.0,76067.0,-0.08,-0.40,30.83,30.30,92.0,-0.01,1.79,12.83,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,0.22,40.0,H-LC,9.94,228592.0,-13332.0,76601.0,-0.66,-5.51,33.51,26.15,8.0,-0.17,1.66,8.02,X40,ATH,IT
47,JIOFIN,387.00,-1.56,40.0,H-LC,14.26,217665.0,259.0,53241.0,-0.26,0.12,24.46,24.61,48.0,0.00,1.58,55.00,XY24,BTT,FINANCE
61,RELIANCE,1533.00,-14.79,41.0,H-LC,7.65,213190.0,2824.0,25967.0,0.01,1.34,12.18,13.68,37.0,0.11,1.55,18.09,XY25,NTT,REFINERIES
43,INFY,2275.00,-19.60,45.0,H-LC,11.30,313856.0,968.0,170706.0,-1.68,0.31,54.39,54.87,3.0,0.01,2.28,6.98,X40,BTT,IT
79,TCS,4998.00,-27.08,47.0,H-LC,14.73,290657.0,-55299.0,179161.0,-0.62,-15.98,61.64,35.80,1.0,-0.31,2.11,2.97,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-6.36,50.0,H-LC,2.37,232178.0,-13973.0,93173.0,-1.60,-5.68,40.13,32.18,16.0,-0.15,1.69,28.32,X200,ATH,IT
55,NESTLEIND,1377.00,-8.48,63.0,H-LC,3.11,281759.0,16333.0,40461.0,0.27,6.15,14.36,21.40,11.0,0.40,2.05,14.42,XY25,NTT,FMCG
45,ITC,452.00,-37.92,51.0,H-LC,3.28,200417.0,279.0,20603.0,0.76,0.14,10.28,10.44,4.0,0.01,1.46,5.71,X40,NTT,FMCG
20,CIPLA,1795.00,-18.07,61.0,H-LC,5.14,218296.0,13796.0,27614.0,1.54,6.75,12.65,20.25,10.0,0.50,1.59,16.07,X40N,BTT,PHARMA
88,VBL,671.64,-10.86,51.0,H-LC,7.47,320914.0,5072.0,107602.0,0.76,1.61,33.53,35.67,5.0,0.05,2.33,15.68,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,16.52,65.0,H-MC,15.65,166425.0,40225.0,5958.0,3.19,31.87,3.58,36.59,108.0,6.75,1.21,48.44,XY24,NTT,HEALTHCARE
52,MASFIN,398.61,-16.44,57.0,H-SC,13.27,94755.0,-3225.0,24826.0,2.90,-3.29,26.20,22.05,152.0,-0.13,0.69,37.73,XR,ATH,FINANCE
62,REPCOHOME,880.00,-61.25,37.0,H-SC,8.46,143220.0,-61778.0,207025.0,2.87,-30.14,144.55,70.85,134.0,-0.30,1.04,17.66,XY24,NTT,FINANCE
60,RELAXO,1176.00,-30.97,63.0,H-SC,6.30,64051.0,-51487.0,86475.0,2.84,-44.56,135.01,30.28,136.0,-0.60,0.47,25.92,X40N,NTT,FOOTWEAR
64,SAIL,228.00,37.93,56.0,M-MC,7.55,217609.0,-7353.0,175001.0,2.66,-3.27,80.42,74.53,192.0,-0.04,1.58,26.34,XY24,BTT,STEEL


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VOLTAS,1918.49,1.16,63.0,H-MC,5.01,212205.0,20463.0,75566.0,-1.80,10.67,35.61,50.08,99.0,0.27,1.54,18.20,XY25,ATH,AC
37,HONAUT,58357.33,-21.28,40.0,H-SC,3.59,110970.0,-16968.0,64107.0,-1.78,-13.26,57.77,36.84,143.0,-0.26,0.81,13.61,X40N,ATH,ELECTRICAL
43,INFY,2275.00,-19.60,45.0,H-LC,11.30,313856.0,968.0,170706.0,-1.68,0.31,54.39,54.87,3.0,0.01,2.28,6.98,X40,BTT,IT
51,LTIM,7230.20,-6.36,50.0,H-LC,2.37,232178.0,-13973.0,93173.0,-1.60,-5.68,40.13,32.18,16.0,-0.15,1.69,28.32,X200,ATH,IT
68,SFL,1287.00,43.03,63.0,M-SC,8.94,188843.0,-73375.0,143200.0,-1.00,-27.98,75.83,26.63,229.0,-0.51,1.37,17.41,XY24,NTT,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,107.68,71.0,M-SC,6.97,89434.0,-11343.0,11439.0,0.11,-11.26,12.79,0.09,245.0,-0.99,0.65,17.50,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,84.36,46.0,H-SC,13.82,116640.0,-25839.0,45361.0,0.30,-18.14,38.89,13.70,163.0,-0.57,0.85,37.67,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.16,55.0,H-SC,3.69,219528.0,-50139.0,86472.0,0.80,-18.59,39.39,13.47,138.0,-0.58,1.60,11.03,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,115.96,52.0,M-SC,3.04,141720.0,-32839.0,32879.0,1.98,-18.81,23.20,0.02,221.0,-1.00,1.03,23.27,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,44.0,H-SC,3.08,114003.0,-29800.0,56545.0,0.01,-20.72,49.60,18.60,149.0,-0.53,0.83,25.81,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-27.08,47.0,H-LC,14.73,290657.0,-55299.0,179161.0,-0.62,-15.98,61.64,35.80,1.0,-0.31,2.11,2.97,X40,BTT,IT
43,INFY,2275.00,-19.60,45.0,H-LC,11.30,313856.0,968.0,170706.0,-1.68,0.31,54.39,54.87,3.0,0.01,2.28,6.98,X40,BTT,IT
45,ITC,452.00,-37.92,51.0,H-LC,3.28,200417.0,279.0,20603.0,0.76,0.14,10.28,10.44,4.0,0.01,1.46,5.71,X40,NTT,FMCG
88,VBL,671.64,-10.86,51.0,H-LC,7.47,320914.0,5072.0,107602.0,0.76,1.61,33.53,35.67,5.0,0.05,2.33,15.68,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,248885.0,-12734.0,131959.0,0.72,-4.87,53.02,45.57,7.0,-0.10,1.81,5.90,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,16.52,65.0,H-MC,15.65,166425.0,40225.0,5958.0,3.19,31.87,3.58,36.59,108.0,6.75,1.21,48.44,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,19.42,186392.0,35479.0,21827.0,0.25,23.51,11.71,37.97,93.0,1.63,1.36,54.76,AR,ATH,AUTO
26,DIXON,18940.15,-29.60,68.0,H-MC,6.78,140824.0,25888.0,10703.0,0.28,22.52,7.60,31.83,81.0,2.42,1.02,45.97,X40N,ATH,IT
27,DMART,5201.00,-7.68,67.0,H-LC,14.31,230491.0,46010.0,24363.0,0.51,24.94,10.57,38.14,38.0,1.89,1.68,38.19,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,16.52,65.0,H-MC,15.65,166425.0,40225.0,5958.0,3.19,31.87,3.58,36.59,108.0,6.75,1.21,48.44,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.85,65.0,H-MC,3.21,194710.0,30040.0,37930.0,0.35,18.24,19.48,41.28,89.0,0.79,1.42,27.68,X40N,ATH,AC
49,KPIGREEN,731.35,5.18,45.0,H-SC,11.09,125433.0,136.0,58866.0,1.16,0.11,46.93,47.09,141.0,0.00,0.91,56.99,MH,ATH,POWER
89,VOLTAS,1918.49,1.16,63.0,H-MC,5.01,212205.0,20463.0,75566.0,-1.80,10.67,35.61,50.08,99.0,0.27,1.54,18.20,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,19.42,186392.0,35479.0,21827.0,0.25,23.51,11.71,37.97,93.0,1.63,1.36,54.76,AR,ATH,AUTO


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,19.42,186392.0,35479.0,21827.0,0.25,23.51,11.71,37.97,93.0,1.63,1.36,54.76,AR,ATH,AUTO
54,NATIONALUM,247.44,-45.32,70.0,H-MC,0.52,108862.0,8358.0,22284.0,2.43,8.32,20.47,30.49,79.0,0.38,0.79,43.42,MH,ATH,METALS
90,WHIRLPOOL,2270.00,-40.66,57.0,M-SC,4.93,100290.0,8792.0,69962.0,0.26,9.61,69.76,86.07,223.0,0.13,0.73,45.64,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.84,44.0,H-SC,12.50,134609.0,11273.0,122494.0,-0.32,9.14,91.00,108.45,119.0,0.09,0.98,33.54,AR,ATH,MISC
91,WIPRO,420.00,-11.98,50.0,M-LC,7.07,154826.0,3881.0,105576.0,-0.48,2.57,68.19,72.51,53.0,0.04,1.13,8.75,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAMMAANCAP,326.00,-172.73,69.0,M-SC,7.26,83550.0,-18660.0,112049.0,-0.08,-18.26,134.11,91.37,208.0,-0.17,0.61,35.63,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,16.23,90666.0,-10297.0,104828.0,2.27,-10.20,115.62,93.63,148.0,-0.10,0.66,35.04,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.41,93389.0,-20160.0,60208.0,2.01,-17.75,64.47,35.27,268.0,-0.33,0.68,128.57,XR,NTT,HOTELS
52,MASFIN,398.61,-16.44,57.0,H-SC,13.27,94755.0,-3225.0,24826.0,2.90,-3.29,26.20,22.05,152.0,-0.13,0.69,37.73,XR,ATH,FINANCE
54,NATIONALUM,247.44,-45.32,70.0,H-MC,0.52,108862.0,8358.0,22284.0,2.43,8.32,20.47,30.49,79.0,0.38,0.79,43.42,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.41,93389.0,-20160.0,60208.0,2.01,-17.75,64.47,35.27,268.0,-0.33,0.68,128.57,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.33,77.0,H-MC,19.42,186392.0,35479.0,21827.0,0.25,23.51,11.71,37.97,93.0,1.63,1.36,54.76,AR,ATH,AUTO
60,RELAXO,1176.00,-30.97,63.0,H-SC,6.30,64051.0,-51487.0,86475.0,2.84,-44.56,135.01,30.28,136.0,-0.60,0.47,25.92,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-7.68,67.0,H-LC,14.31,230491.0,46010.0,24363.0,0.51,24.94,10.57,38.14,38.0,1.89,1.68,38.19,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,12.85,65.0,H-MC,3.21,194710.0,30040.0,37930.0,0.35,18.24,19.48,41.28,89.0,0.79,1.42,27.68,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.21
1,25,42.44
2,50,72.06


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.49
SC    36.51
MC    26.01
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.79
X40      15.06
X40N     13.97
XY25     11.24
AR        9.18
XR        8.82
OX40N     7.23
X5K       2.26
MH        1.70
X200      1.69
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.49
H-MC    23.13
H-SC    21.89
M-SC    13.05
M-LC     6.97
M-MC     2.56
L-SC     1.57
L-LC     1.03
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,1.03,31.21
IT,12.98,-15.62,72.97
FINANCE,7.18,-22.14,67.18
BANKS,7.15,-15.94,65.17
PAINTS,5.86,-14.64,40.30
MISC,5.72,-16.73,83.64
ELECTRICAL,5.57,-7.50,66.76
AUTO,4.49,-9.71,57.44
AC,3.67,4.07,28.41


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2812951.0,23
AR,1218043.0,10
XR,1081298.0,13
X40,893950.0,10
X40N,514832.0,11
OX40N,465016.0,10
XY25,433800.0,8
SR,196625.0,2
X5K,128815.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2767189.0,27
M-SC,1449649.0,17
H-LC,1414576.0,18
H-MC,1194220.0,18
M-LC,420292.0,5
M-MC,325916.0,2
L-SC,245806.0,3
L-MC,59202.0,1
L-LC,42803.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,931031.0,6
M-SC,XY24,859868.0,7
H-SC,AR,664805.0,5
H-LC,X40,596716.0,5
H-SC,XR,522546.0,6
H-MC,XY24,466398.0,5
H-LC,AR,371121.0,2
M-MC,XY24,325916.0,2
H-MC,X40,218720.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
